# Chatbots

In [ ]:
from huggingface_hub import login
token = ""
print("Hugging Face logging")
login(token)

In [ ]:
import torch
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")


## Creación de una clase para chatbot básico

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class Chatbot:

    def __init__(self, chatbot_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", cache_dir="./models/TinyLlama-1.1B-Chat-v1.0"):
        global device
        self.model_device = device
        self.tokenizer = AutoTokenizer.from_pretrained(chatbot_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            chatbot_model,
            torch_dtype = torch.float32 if device == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False
        ).to(device)


    def __run_prompt(self, prompt, do_sample, temperature, top_p, show_prompt):
        formatted_prompt = self.tokenizer.apply_chat_template(conversation=prompt, tokenize=False, return_dict=False, add_generation_prompt=True)

        # Tokenizar
        inputs = self.tokenizer(formatted_prompt, truncation=False,return_tensors="pt")
        inputs = {k: v.to(self.model_device) for k, v in inputs.items()}

        # Muestra infomacion de log
        if show_prompt:
          print(formatted_prompt)
          print("--- Token size: ---")
          [print("\t", k, ": ", len(v[0])) for k, v in inputs.items()]
          print("-------------------")

        # Generar respuesta
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=250,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=self.tokenizer.eos_token_id
        )

        # Decodificar y limpiar respuesta
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def __format_prompt(self, user_prompt, system_prompt):
        return [
            { "role": "system", "content": system_prompt,},
            { "role": "user", "content":  user_prompt},
        ]


    def answer(self, user_prompt, system_prompt, do_sample=True, temperature=0.1,
               top_p=0.9, show_prompt=False):
        prompt = self.__format_prompt(user_prompt, system_prompt)
        return self.__run_prompt(prompt, do_sample, temperature, top_p, show_prompt)


In [ ]:
chatbot = Chatbot()

answer = chatbot.answer(user_prompt="How many helicopters can a human eat in one sitting?", system_prompt="You are a friendly chatbot who always responds in the style of a pirate", show_prompt=True)

print(answer)


#### Tarea CB1

Vamos a consultar al modelo para que nos resuelva una duda de programación, en particular, `El siguiente código python da un error, podrías arreglarlo por mi: x=1; y='1'; z = x+y`. ¿Cúal sería un buen prompt para el sistema?
Generar distintas respuestas variando la temperatura, usar 0.1, 0.7, 1, y 1.3

In [ ]:
# TODO: usar temperatura 0.1

In [ ]:
# TODO: usar temperatura 0.7

In [ ]:
# TODO: usar temperatura 1

In [ ]:
# TODO: usar temperatura 1.3

#### Tarea CB2

En los ejemplos anteriores la interacción con el chatbot finaliza al terminar la ejecución, vamos a construir una interacción humano-máquina perpetua. Para ello, escriba el código necesario para pedirle al usuario el prompt necesario, pasárselo al modelo, e imprimir su respuesta. Este comportamiento tiene que repetirse sin parar.

In [ ]:
# TODO: código de prompting perpetuo


## Creación de una clase para chatbot avanzado

#### Tarea CB3

En el código anterior, las respuestas que nos proporciona el modelo no se tienen en cuenta para la siguiente interacción. Vamos a generar una estructura de datos para que se vayan guardando en la conversación. Para ello modifique la clase `Chatbot`.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class DialogueController:

    def __init__(self, system_prompt, assistant_token="<|assistant|>"):
        # TODO: crear un atributo prompt e inicializarlo con la parte correspondiente al sistema
        pass

    def get_prompt(self):
        # TODO: devolver el prompt
        pass
    def add_user_prompt(self, user_prompt, role="user"):
        # TODO: añadir el nuevo prompt del usuario
        # TODO: devolver el prompt actualizado
        pass

    def add_assistant_prompt(self, assistant_prompt):
        # TODO: añadir el nuevo prompt del asistente
        # TODO: hay que tener en cuenta la estructura del assistant_prompt
        pass


class Chatbot2:

    def __init__(self, controller, device_setup, chatbot_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", cache_dir="./models/TinyLlama-1.1B-Chat-v1.0"):
        self.tokenizer = AutoTokenizer.from_pretrained(chatbot_model)
        self.model = AutoModelForCausalLM.from_pretrained(
            chatbot_model,
            torch_dtype = torch.float32 if device_setup == "mps" else (torch.float16 if torch.cuda.is_available() else torch.float32),
            cache_dir=cache_dir,
            local_files_only=False
        ).to(device_setup)
        self.model_device = device_setup
        self.dialogue_controller = controller

    def __run_prompt(self, prompt, do_sample, temperature, top_p, show_prompt):
        formatted_prompt = self.tokenizer.apply_chat_template(conversation=prompt, tokenize=False, return_dict=False, add_generation_prompt=True)

        # Tokenizar
        inputs = self.tokenizer(formatted_prompt, return_tensors="pt")
        inputs = {k: v.to(self.model_device) for k, v in inputs.items()}

        # Muestra infomacion de log
        if show_prompt:
          print(formatted_prompt)
          print("--- Token size: ---")
          [print("\t", k, ": ", len(v[0])) for k, v in inputs.items()]
          print("-------------------")

        # Generar respuesta
        outputs = self.model.generate(
            **inputs,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=self.tokenizer.eos_token_id
        )

        # Decodificar y limpiar respuesta
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


    def answer(self, user_prompt, do_sample=True, temperature=0.1, top_p=0.9, show_prompt=False):
        # Actualiza el prompt
        prompt = self.dialogue_controller.add_user_prompt(user_prompt)
        # Resolver prompt
        answer = self.__run_prompt(prompt, do_sample, temperature, top_p, show_prompt)
        # Actualiza el prompt con la respuesta del asistente
        self.dialogue_controller.add_assistant_prompt(answer)
        return answer



In [ ]:
answer = "Please ask something: "
dialog_controller = DialogueController(system_prompt="Responde mis dudas como un chatbot educado de manera coincisa")
chatbot_windowing = Chatbot2(controller=dialog_controller, device_setup=device)

while True:
    user_input = input(answer)
    answer = chatbot_windowing.answer(user_prompt=user_input, show_prompt=True, temperature=0.1)
    print("----------------------")


#### Tarea CB4

Ejecute el código de la celda anterior y utilice el prompt del fichero `./provided/prompt-overflow.txt`. ¿Qué ha ocurrido? ¿Qué habría que hacer para evitar esta situación?

In [ ]:
# TODO: Modificar el tokenizador para que trunque el prompt y conozca el tamaño máximo de prompt, añadir esa información también al modelo cuando genera su output